# Library Imports

In [1]:
import os
import numpy as np
import pandas as pd
from scipy import io
# from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score, confusion_matrix, f1_score,\
                            precision_score, recall_score, accuracy_score,\
                            average_precision_score, precision_recall_curve
from imblearn.over_sampling import RandomOverSampler
from src.helper_functions import load_data, get_model_perfs, init_model_perfs,\
                                 save_model, save_model_perfs, check_is_best,\
                                 read_model, evaluate_model_predictions,\
                                 update_model_perfs, check_and_save,\
                                 adjusted_classes

Using TensorFlow backend.
/Users/sean/anaconda3/envs/metis/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/sean/anaconda3/envs/metis/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/sean/anaconda3/envs/metis/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/sean/anaconda3/envs

In [2]:
import keras
from keras.callbacks import Callback
from keras.layers import Dense, Activation, Dropout
keras.__version__

'2.2.5'

# Load Data

In [3]:
x_tr, y_tr, x_te, y_te = load_data()

# Build Neural Network

Following the desciption in section 2.2.4 of the [DeepTox article](https://www.frontiersin.org/articles/10.3389/fenvs.2015.00080/full), I tried to use intermediate values in [Table 2](https://www.frontiersin.org/articles/10.3389/fenvs.2015.00080/full#T2) to build the neural network:

Following [this question/answer](https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model) and [this question/answer](https://stackoverflow.com/questions/54065733/how-to-employ-the-scikit-learn-evaluation-metrics-functions-with-keras-in-python) to implement usage of recall in model training:

In [4]:
drop_out=0.5    # DeepTox range: 0.5, 0.2, 0
L2_reg = 0.0001 # Default = 0.01
layers = 3      # DeepTox range: 1, 2, 3, 4
act = 'sigmoid' # Consider sigmoid and tanh
neurons = 1024  # DeepTox range: 1024, 2048, 4096, 8192, 16384
# Info on decay: https://datascience.stackexchange.com/questions/26112/decay-parameter-in-keras-optimizers
decay = 0       # DeepTox range: 10^-4, 10^-5, 10^-6
learn_rate = 0.1  #Research appropriate range
DNN = keras.Sequential()
DNN.add(Dense(neurons, activation=act,input_shape=x_tr.shape[1:],name='h0_'+act+'_activation'))
DNN.add(Dropout(rate=drop_out,name='Dropout0'))
for i in range(1,layers):
    DNN.add(Dense(units=neurons, activation=act,\
                  name='h'+str(i)+'_'+act+'_activation',\
                  kernel_regularizer=keras.regularizers.l2(L2_reg)))
    DNN.add(Dropout(rate=drop_out,name='Dropout'+str(i)))
DNN.add(Dense(units=1, activation='sigmoid'))
keras.optimizers.Adam(lr=learn_rate, beta_1=0.9,\
                      beta_2=0.999, decay=decay, amsgrad=False)
DNN.compile(optimizer='adam', loss='binary_crossentropy',\
            metrics=['accuracy'])
DNN.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
h0_sigmoid_activation (Dense (None, 1024)              1684480   
_________________________________________________________________
Dropout0 (Dropout)           (None, 1024)              0         
_________________________________________________________________
h1_sigmoid_activation (Dense (None, 1024)              1049600   
_________________________________________________________________
Dropout1 (Dropout)           (None, 1024)              0         
_________________________________________________________________
h2_sigmoid_activation (Dense (None, 1024)              1049600   
_________________________________________________________________

# Train DNN's

In [5]:
for target in y_tr.columns:
    # Determine rows with available data
    rows_tr = np.isfinite(y_tr[target]).values
    rows_te = np.isfinite(y_te[target]).values
    x,y = x_tr[rows_tr], y_tr[target][rows_tr]
    
    # Address Class Imbalance
    x_train, x_val, y_train, y_val = train_test_split(x, y, stratify=y, \
                                                      test_size=0.2, random_state=42)
    ros = RandomOverSampler(random_state=0)
    x_resampled, y_resampled = ros.fit_sample(x_train,y_train)
    
    # Train the DNN
    DNN.fit(
        x_resampled, y_resampled, batch_size=512, epochs=100,\
        validation_data=(x_val,y_val), verbose=1,
        callbacks=[
            keras.callbacks.EarlyStopping(patience=16,verbose=1,\
                                          restore_best_weights=True),
            keras.callbacks.ReduceLROnPlateau(factor=0.5,patience=3,verbose=1)
        ])
    
    # Get predictions, calculate model performance and save info
    p_te = DNN.predict(x_te[rows_te])[:,0]
    y_testing=y_te[target][rows_te]
    auc_te = roc_auc_score(y_testing, p_te)
    print("%15s: %3.5f" % (target, auc_te))

    y_hat_testing=DNN.predict_classes(x_te[rows_te])
    average_precision=average_precision_score(y_testing,p_te)
    mv=evaluate_model_predictions(target,'DNN',0.5,y_testing,y_hat_testing,\
                                  auc_te,average_precision)
    check_and_save(target,mv,DNN,True)
    
    # Find max F1 varying probability threshold, calculate modified performance, save
    precision, recall, thresholds = precision_recall_curve(y_testing,p_te)
    # f1 = 2*precision*recall/(precision+recall)  # Sometimes precision=recall=0!
    p,r,t = zip(*[i for i in zip(precision,recall,np.append(thresholds,1)) if i[0:2]!=(0,0)])
    p,r,t = np.array(p),np.array(r),np.array(t)
    f1 = 2*p*r/(p+r)
    m_idx = np.argmax(f1)
    m_thresh = thresholds[m_idx]
    y_hat_testing_adj=adjusted_classes(p_te,m_thresh)
    mv=evaluate_model_predictions(target,'DNN_modT',m_thresh,y_testing,y_hat_testing_adj,\
                                  auc_te,average_precision)
    check_and_save(target,mv,DNN,True)
    display(get_model_perfs(target))

Instructions for updating:
Use tf.cast instead.
Train on 11934 samples, validate on 1689 samples
Epoch 1/100
11934/11934 [==============================] - 3s 227us/step - loss: 1.1133 - acc: 0.5215 - val_loss: 0.7070 - val_acc: 0.8840
Epoch 2/100
11934/11934 [==============================] - 2s 175us/step - loss: 0.8746 - acc: 0.5747 - val_loss: 0.8319 - val_acc: 0.4938
Epoch 3/100
11934/11934 [==============================] - 2s 181us/step - loss: 0.7788 - acc: 0.6629 - val_loss: 0.7846 - val_acc: 0.5595
Epoch 4/100
11934/11934 [==============================] - 3s 210us/step - loss: 0.7400 - acc: 0.6865 - val_loss: 0.6602 - val_acc: 0.7105
Epoch 5/100
11934/11934 [==============================] - 3s 210us/step - loss: 0.7156 - acc: 0.6886 - val_loss: 0.5235 - val_acc: 0.8502
Epoch 6/100
11934/11934 [==============================] - 2s 200us/step - loss: 0.6984 - acc: 0.7056 - val_loss: 0.8266 - val_acc: 0.5317
Epoch 7/100
11934/11934 [==============================] - 2s 201us/s

,model,threshold,accuracy,precision,recall,f1,auc_roc,avg_precision,confusion_matrix,model_filename
0,RF,0.500000,0.911475,0.756757,0.383562,0.509091,0.905028,0.602102,"[[528, 9], [45, 28]]",RF0.joblib
1,RF_modT,0.235000,0.842623,0.424837,0.890411,0.575221,0.905028,0.602102,"[[449, 88], [8, 65]]",RF_modT0.joblib
2,DNN,0.500000,0.814754,0.364865,0.739726,0.488688,0.864723,0.524576,"[[443, 94], [19, 54]]",DNN0.h5
3,DNN_modT,0.773516,0.901639,0.603175,0.520548,0.558824,0.864723,0.524576,"[[512, 25], [35, 38]]",DNN_modT0.h5


Train on 14834 samples, validate on 1931 samples
Epoch 1/100
14834/14834 [==============================] - 3s 221us/step - loss: 1.0468 - acc: 0.4847 - val_loss: 0.9818 - val_acc: 0.4443
Epoch 2/100
14834/14834 [==============================] - 3s 221us/step - loss: 0.9139 - acc: 0.5450 - val_loss: 0.9631 - val_acc: 0.4459
Epoch 3/100
14834/14834 [==============================] - 3s 215us/step - loss: 0.8575 - acc: 0.5810 - val_loss: 0.8899 - val_acc: 0.4858
Epoch 4/100
14834/14834 [==============================] - 3s 217us/step - loss: 0.8338 - acc: 0.5866 - val_loss: 0.8453 - val_acc: 0.5049
Epoch 5/100
14834/14834 [==============================] - 3s 217us/step - loss: 0.8115 - acc: 0.6066 - val_loss: 0.8269 - val_acc: 0.5308
Epoch 6/100
14834/14834 [==============================] - 3s 209us/step - loss: 0.7909 - acc: 0.6201 - val_loss: 0.8087 - val_acc: 0.5526
Epoch 7/100
14834/14834 [==============================] - 3s 212us/step - loss: 0.7809 - acc: 0.6283 - val_loss: 0.7

,model,threshold,accuracy,precision,recall,f1,auc_roc,avg_precision,confusion_matrix,model_filename
0,RF,0.500000,0.981229,0.666667,0.166667,0.266667,0.678934,0.232723,"[[573, 1], [10, 2]]",RF0.joblib
1,RF_modT,0.730000,0.982935,1.000000,0.166667,0.285714,0.678934,0.232723,"[[574, 0], [10, 2]]",RF_modT0.joblib
2,DNN,0.500000,0.721843,0.042424,0.583333,0.079096,0.741507,0.168663,"[[416, 158], [5, 7]]",DNN0.h5
3,DNN_modT,0.792447,0.965870,0.250000,0.333333,0.285714,0.741507,0.168663,"[[562, 12], [8, 4]]",DNN_modT0.h5


Train on 13670 samples, validate on 1771 samples
Epoch 1/100
13670/13670 [==============================] - 3s 207us/step - loss: 0.6926 - acc: 0.7134 - val_loss: 0.5940 - val_acc: 0.7493
Epoch 2/100
13670/13670 [==============================] - 3s 214us/step - loss: 0.6916 - acc: 0.7143 - val_loss: 0.5947 - val_acc: 0.7493
Epoch 3/100
13670/13670 [==============================] - 3s 212us/step - loss: 0.6977 - acc: 0.7115 - val_loss: 0.5955 - val_acc: 0.7487
Epoch 4/100
13670/13670 [==============================] - 3s 211us/step - loss: 0.6946 - acc: 0.7108 - val_loss: 0.5961 - val_acc: 0.7487

Epoch 00004: ReduceLROnPlateau reducing learning rate to 3.051757957450718e-08.
Epoch 5/100
13670/13670 [==============================] - 3s 210us/step - loss: 0.6950 - acc: 0.7102 - val_loss: 0.5964 - val_acc: 0.7487
Epoch 6/100
13670/13670 [==============================] - 3s 212us/step - loss: 0.6970 - acc: 0.7123 - val_loss: 0.5967 - val_acc: 0.7487
Epoch 7/100
13670/13670 [===========

...

,model,threshold,accuracy,precision,recall,f1,auc_roc,avg_precision,confusion_matrix,model_filename
0,RF,0.500000,0.965574,0.600000,0.136364,0.222222,0.759586,0.239563,"[[586, 2], [19, 3]]",RF0.joblib
1,RF_modT,0.340000,0.965574,0.545455,0.272727,0.363636,0.759586,0.239563,"[[583, 5], [16, 6]]",RF_modT0.joblib
2,DNN,0.500000,0.726230,0.055215,0.409091,0.097297,0.626739,0.054621,"[[434, 154], [13, 9]]",DNN0.h5
3,DNN_modT,0.450345,0.708197,0.066667,0.545455,0.118812,0.626739,0.054621,"[[420, 168], [10, 12]]",DNN_modT0.h5


Train on 10204 samples, validate on 1512 samples
Epoch 1/100
10204/10204 [==============================] - 2s 201us/step - loss: 0.7619 - acc: 0.6424 - val_loss: 0.6140 - val_acc: 0.7421
Epoch 2/100
10204/10204 [==============================] - 2s 218us/step - loss: 0.7654 - acc: 0.6400 - val_loss: 0.6140 - val_acc: 0.7421
Epoch 3/100
10204/10204 [==============================] - 2s 215us/step - loss: 0.7604 - acc: 0.6392 - val_loss: 0.6140 - val_acc: 0.7421
Epoch 4/100
10204/10204 [==============================] - 2s 219us/step - loss: 0.7587 - acc: 0.6449 - val_loss: 0.6140 - val_acc: 0.7421

Epoch 00004: ReduceLROnPlateau reducing learning rate to 2.7755576933948015e-20.
Epoch 5/100
10204/10204 [==============================] - 2s 214us/step - loss: 0.7617 - acc: 0.6446 - val_loss: 0.6140 - val_acc: 0.7421
Epoch 6/100
10204/10204 [==============================] - 2s 213us/step - loss: 0.7611 - acc: 0.6400 - val_loss: 0.6140 - val_acc: 0.7421
Epoch 7/100
10204/10204 [==========

,model,threshold,accuracy,precision,recall,f1,auc_roc,avg_precision,confusion_matrix,model_filename
0,RF,0.500000,0.906077,0.615385,0.40,0.484848,0.930331,0.560184,"[[468, 15], [36, 24]]",RF0.joblib
1,RF_modT,0.382500,0.913444,0.600000,0.65,0.624000,0.930331,0.560184,"[[457, 26], [21, 39]]",RF_modT0.joblib
2,DNN,0.500000,0.815838,0.330508,0.65,0.438202,0.803882,0.296565,"[[404, 79], [21, 39]]",DNN0.h5
3,DNN_modT,0.505391,0.817680,0.333333,0.65,0.440678,0.803882,0.296565,"[[405, 78], [21, 39]]",DNN_modT0.h5


Train on 13340 samples, validate on 1781 samples
Epoch 1/100
13340/13340 [==============================] - 3s 206us/step - loss: 0.7926 - acc: 0.6395 - val_loss: 0.6058 - val_acc: 0.7344
Epoch 2/100
13340/13340 [==============================] - 3s 218us/step - loss: 0.7896 - acc: 0.6397 - val_loss: 0.6058 - val_acc: 0.7344
Epoch 3/100
13340/13340 [==============================] - 3s 215us/step - loss: 0.7886 - acc: 0.6389 - val_loss: 0.6058 - val_acc: 0.7344
Epoch 4/100
13340/13340 [==============================] - 3s 214us/step - loss: 0.7935 - acc: 0.6389 - val_loss: 0.6058 - val_acc: 0.7344

Epoch 00004: ReduceLROnPlateau reducing learning rate to 8.673617791858755e-22.
Epoch 5/100
13340/13340 [==============================] - 3s 213us/step - loss: 0.7917 - acc: 0.6403 - val_loss: 0.6058 - val_acc: 0.7344
Epoch 6/100
13340/13340 [==============================] - 3s 212us/step - loss: 0.7906 - acc: 0.6421 - val_loss: 0.6058 - val_acc: 0.7344
Epoch 7/100
13340/13340 [===========

,model,threshold,accuracy,precision,recall,f1,auc_roc,avg_precision,confusion_matrix,model_filename
0,RF,0.500000,0.933442,0.500000,0.024390,0.046512,0.806702,0.221990,"[[574, 1], [40, 1]]",RF0.joblib
1,RF_modT,0.220000,0.886364,0.254237,0.365854,0.300000,0.806702,0.221990,"[[531, 44], [26, 15]]",RF_modT0.joblib
2,DNN,0.500000,0.753247,0.150943,0.585366,0.240000,0.768696,0.168234,"[[440, 135], [17, 24]]",DNN0.h5
3,DNN_modT,0.540362,0.780844,0.169014,0.585366,0.262295,0.768696,0.168234,"[[457, 118], [17, 24]]",DNN_modT0.h5


# Performance Metrics Saving Development

In [ ]:
p_te = DNN.predict(x_te[rows_te])[:,0]
y_testing=y_te[target][rows_te]
auc_te = roc_auc_score(y_testing, p_te)
print("%15s: %3.5f" % (target, auc_te))

y_hat_testing=DNN.predict_classes(x_te[rows_te])
average_precision=average_precision_score(y_testing,p_te)
mv=evaluate_model_predictions(target,'DNN',0.5,y_testing,y_hat_testing,\
                              auc_te,average_precision)
#     check_and_save(target,mv,DNN,True)
display(mv)
precision, recall, thresholds = precision_recall_curve(y_testing,p_te)
# f1 = 2*precision*recall/(precision+recall)
p,r,t = zip(*[i for i in zip(precision,recall,np.append(thresholds,1)) if i[0:2]!=(0,0)])
p,r,t = np.array(p),np.array(r),np.array(t)
f1 = 2*p*r/(p+r)
m_idx = np.argmax(f1)
m_thresh = thresholds[m_idx]
y_hat_testing_adj=adjusted_classes(p_te,m_thresh)
mv=evaluate_model_predictions(target,'DNN_modT',m_thresh,y_testing,y_hat_testing_adj,\
                              auc_te,average_precision)
#     check_and_save(target,mv,DNN,True)
# display(get_model_perfs(target))
display(mv)

# Troubleshooting Precision-Recall Error

In [ ]:
f1 = 2*precision*recall/(precision+recall)

In [ ]:
a, b = zip(*zip(precision,recall))
type(a)

In [ ]:
p,r,t = zip(*[i for i in zip(precision,recall,np.append(thresholds,1)) if i[0:2]!=(0,0)])
p,r,t = np.array(p),np.array(r),np.array(t)
f1 = 2*p*r/(p+r)

# Testing DNN Model Saving.

In [6]:
filename = save_model(target,'DNN_test',DNN,True)
filename

'DNN_test0.h5'

In [7]:
read_model(target,filename)

In [8]:
keras.__version__

'2.2.5'